# Load Margin Analysis using Grid2Op

## TFG - Pablo Pedrosa Prats
### ICAI - Universidad Pontificia Comillas

---

**Objetivo:** Aproximar el margen de carga de un sistema eléctrico incrementando la demanda de forma escalonada y observando el comportamiento del sistema en simulación.

**Metodología:**
1. Aplicar un factor de escalado de demanda λ (P y Q)
2. Ejecutar un flujo de potencia para cada paso
3. Registrar variables relevantes (tensión mínima, cargas de líneas, etc.)
4. Definir λ* como el mayor λ para el que el sistema permanece operable

**Criterios de límite (no operable):**
- No convergencia del flujo de potencia
- Tensiones fuera de rango (0.9-1.1 p.u.)
- Sobrecargas térmicas > 100%

In [ ]:
# Importaciones
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Grid2Op
import grid2op
from grid2op.PlotGrid import PlotMatplot

# Módulos propios
from src.load_margin import LoadMarginAnalyzer, LoadMarginResult, create_results_dataframe
from src.agents.greedy_agent import GreedyAgent, GreedyLoadMarginOptimizer
from src.agents.sensitivity_agent import SensitivityAgent, SensitivityLoadMarginOptimizer
from src.visualization import (
    plot_load_margin_curve, 
    plot_n1_comparison, 
    plot_method_comparison,
    plot_pv_curve,
    create_summary_table,
    generate_full_report
)

print(f"Grid2Op version: {grid2op.__version__}")

## 1. Configuración del Entorno

Utilizamos el entorno `l2rpn_case14_sandbox` que es una red de 14 buses basada en el caso IEEE 14.

In [ ]:
# Crear entorno Grid2Op
env_name = "l2rpn_case14_sandbox"
env = grid2op.make(env_name)

print(f"Entorno: {env_name}")
print(f"Número de líneas: {env.n_line}")
print(f"Número de subestaciones: {env.n_sub}")
print(f"Número de generadores: {env.n_gen}")
print(f"Número de cargas: {env.n_load}")

In [ ]:
# Visualizar la red inicial
obs = env.reset()

plot_helper = PlotMatplot(env.observation_space)
fig = plot_helper.plot_obs(obs, line_info="rho", load_info="p", gen_info="p")
plt.title("Red Eléctrica - Estado Inicial")
plt.show()

In [ ]:
# Información del estado inicial
print("=" * 50)
print("ESTADO INICIAL DEL SISTEMA")
print("=" * 50)
print(f"\nDemanda total: {np.sum(obs.load_p):.2f} MW")
print(f"Generación total: {np.sum(obs.gen_p):.2f} MW")
print(f"Pérdidas: {np.sum(obs.gen_p) - np.sum(obs.load_p):.2f} MW")
print(f"\nCarga máxima de línea (ρ_max): {np.max(obs.rho):.2%}")
print(f"Línea más cargada: {np.argmax(obs.rho)}")

# Voltajes
v_all = np.concatenate([obs.v_or[obs.v_or > 0], obs.v_ex[obs.v_ex > 0]])
print(f"\nRango de voltajes: [{np.min(v_all):.2f}, {np.max(v_all):.2f}] kV")

## 2. Análisis de Margen de Carga - Caso Base

Incrementamos la demanda progresivamente hasta encontrar el punto de colapso.

In [ ]:
# Crear analizador de margen de carga
analyzer = LoadMarginAnalyzer(
    env=env,
    v_min_pu=0.9,    # Límite inferior de tensión
    v_max_pu=1.1,    # Límite superior de tensión
    rho_max=1.0      # Límite de sobrecarga (100%)
)

print("Analizador configurado.")
print(f"Límites de tensión: [{analyzer.v_min_pu}, {analyzer.v_max_pu}] p.u.")
print(f"Límite de sobrecarga: {analyzer.rho_max:.0%}")

In [ ]:
# Calcular margen de carga del caso base
result_base = analyzer.calculate_load_margin(
    lambda_start=1.0,
    lambda_end=1.5,
    lambda_step=0.01,
    verbose=True
)

In [ ]:
# Visualizar resultados del caso base
fig = plot_load_margin_curve(
    result_base,
    title=f"Caso Base - Margen de Carga λ* = {result_base.lambda_max:.3f}"
)
plt.show()

In [ ]:
# Curva PV (curva de nariz)
fig = plot_pv_curve(
    result_base,
    title="Curva PV - Análisis de Estabilidad de Voltaje"
)
plt.show()

## 3. Análisis de Contingencias N-1

Evaluamos el margen de carga bajo diferentes contingencias (desconexión de líneas).

In [ ]:
# Seleccionar líneas para análisis N-1
# Analizar las 5 líneas más cargadas
obs = env.reset()
most_loaded_lines = np.argsort(obs.rho)[-5:][::-1]
print(f"Líneas más cargadas: {most_loaded_lines}")
print(f"Cargas (ρ): {obs.rho[most_loaded_lines]}")

In [ ]:
# Ejecutar análisis N-1
n1_results = analyzer.run_n1_analysis(
    line_ids=most_loaded_lines.tolist(),
    lambda_end=1.5,
    lambda_step=0.01,
    verbose=True
)

In [ ]:
# Visualizar comparación N-1
fig = plot_n1_comparison(
    n1_results,
    title="Análisis de Contingencias N-1"
)
plt.show()

In [ ]:
# Tabla resumen de resultados
df_n1 = create_summary_table(n1_results)
display(df_n1)

## 4. Control Topológico - Agente Greedy (Baseline)

Implementamos un agente Greedy con Look-Ahead que busca acciones correctivas cuando hay sobrecargas.

In [ ]:
# Crear optimizador Greedy
greedy_optimizer = GreedyLoadMarginOptimizer(
    env=env,
    rho_threshold=0.9,
    lookahead_steps=3
)

print("Optimizador Greedy configurado.")

In [ ]:
# Optimizar margen de carga con agente Greedy
greedy_results = greedy_optimizer.optimize_load_margin(
    lambda_start=1.0,
    lambda_end=1.5,
    lambda_step=0.01,
    max_actions=10,
    verbose=True
)

print(f"\nMargen de carga con Greedy: λ* = {greedy_results['lambda_max']:.3f}")
print(f"Acciones tomadas: {len(greedy_results['actions_taken'])}")

## 5. Control Topológico - Agente basado en Sensibilidades (PTDF/LODF)

Implementamos un agente más sofisticado que utiliza análisis de sensibilidades para filtrar el espacio de acciones.

In [ ]:
# Crear optimizador basado en sensibilidades
sensitivity_optimizer = SensitivityLoadMarginOptimizer(
    env=env,
    top_k_candidates=5,
    rho_threshold=0.9
)

print("Optimizador de Sensibilidades configurado.")

In [ ]:
# Optimizar margen de carga con análisis de sensibilidades
sensitivity_results = sensitivity_optimizer.optimize_load_margin(
    lambda_start=1.0,
    lambda_end=1.5,
    lambda_step=0.01,
    max_actions=10,
    verbose=True
)

print(f"\nMargen de carga con Sensibilidades: λ* = {sensitivity_results['lambda_max']:.3f}")
print(f"Acciones tomadas: {len(sensitivity_results['actions_taken'])}")

## 6. Comparación de Métodos

In [ ]:
# Comparación visual de los tres métodos
fig = plot_method_comparison(
    base_result=result_base,
    greedy_result=greedy_results,
    sensitivity_result=sensitivity_results,
    title="Comparación de Métodos de Control Topológico"
)
plt.show()

In [ ]:
# Resumen de resultados
print("=" * 60)
print("RESUMEN DE RESULTADOS")
print("=" * 60)
print(f"\n{'Método':<25} {'λ*':<10} {'Mejora':<15} {'Acciones':<10}")
print("-" * 60)

base_lambda = result_base.lambda_max
print(f"{'Caso Base':<25} {base_lambda:<10.3f} {'-':<15} {0:<10}")

greedy_lambda = greedy_results['lambda_max']
greedy_improvement = (greedy_lambda - base_lambda) / base_lambda * 100
print(f"{'Greedy Agent':<25} {greedy_lambda:<10.3f} {greedy_improvement:+.1f}%{'':>8} {len(greedy_results['actions_taken']):<10}")

sens_lambda = sensitivity_results['lambda_max']
sens_improvement = (sens_lambda - base_lambda) / base_lambda * 100
print(f"{'Sensitivity Agent':<25} {sens_lambda:<10.3f} {sens_improvement:+.1f}%{'':>8} {len(sensitivity_results['actions_taken']):<10}")

## 7. Generación del Informe Completo

In [ ]:
# Generar todas las gráficas y guardar
df_summary = generate_full_report(
    base_results=n1_results,
    greedy_results=greedy_results,
    sensitivity_results=sensitivity_results,
    output_dir="../results"
)

print("\nTabla resumen final:")
display(df_summary)

## 8. Conclusiones

### Hallazgos Principales:

1. **Margen de Carga Base**: El sistema puede soportar un incremento de demanda de λ* antes de violar límites operativos.

2. **Contingencias N-1**: Las contingencias más severas son [identificar líneas críticas], que reducen significativamente el margen de carga.

3. **Control Topológico**:
   - El agente Greedy mejora el margen en X%
   - El agente basado en Sensibilidades mejora el margen en Y%
   - El enfoque de sensibilidades es más eficiente computacionalmente

### Limitaciones:

- Modelo DC simplificado para cálculo de PTDF/LODF
- No se consideran límites de rampa de generadores
- Análisis estático (no dinámico)

### Trabajo Futuro:

- Implementar cálculo exacto de PTDF/LODF desde matriz de admitancia
- Considerar refuerzos de red (aumento de capacidad)
- Extender a redes más grandes (IEEE 118, casos reales)